In [240]:
import os 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import ElasticNetCV, ElasticNet
from sklearn.metrics import mean_squared_error, explained_variance_score

# Results Table

In [244]:
os.chdir("/Users/simonrdkn/Documents/commalla_acoustic_analysis/feature sets")
original_features = pd.read_csv("original feature extraction.csv")
sauter_features = pd.read_csv("sauter_feature_set.csv")

results_table = pd.DataFrame({
    "dimension":[],
    "dataset":[],
    "technique":[],
    "r2":[],
    "MSE":[],
    "explained variance":[]
})

feature_weights = pd.DataFrame({
    "dimension":[],
    "dataset":[],
    "technique":[],
    "feature":[],
    "coefficent":[],
    "rank":[]
})


In [245]:
raw_datasets = [sauter_features, original_features]
dataset_names = ["Sauter Features", "Original Features"]
techniques = [ElasticNetCV(cv=5, l1_ratio=0.5), LassoCV(cv=5)]
technique_names = ["Elastic Net", "Lasso"]
# # Use param_grids when using regression_cv = GridSearchCV(techniques[j], param_grids[j], cv=5), switch LassoCV() to Lasso()
# param_grids = [{'l1_ratio': [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}, {'alpha': [0.001, 0.01, 0.1, 1, 10,]}]

for dimension in ["mean_arousal", "mean_valence"]:
    for i in range(2):
        for j in range(2):
            dataset = raw_datasets[i].dropna()

            X = dataset[dataset.columns[7:]] # check columns if results are weird 
            y = dataset[dimension]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)
            scaler = StandardScaler() # scaling the data
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            regression_cv = techniques[j] # couldn't get feature weights from GridSearchCV()
            regression_cv.fit(X_train, y_train)
            y_pred = regression_cv.predict(X_test)
            # print(dimension, dataset_names[i], technique_names[j], regression_cv.score(X_test, y_test), mean_squared_error(y_test, y_pred), explained_variance_score(y_test, y_pred))

            new_row = pd.DataFrame({
                    "dimension":[dimension],
                    "dataset":[dataset_names[i]],
                    "technique":[technique_names[j]], 
                    "r2":[regression_cv.score(X_test, y_test)],
                    "MSE":[mean_squared_error(y_test, y_pred)],
                    "explained variance": [explained_variance_score(y_test, y_pred)]
                    })

            results_table = pd.concat([results_table, new_row])

            zipped = list(zip(X.columns, regression_cv.coef_))
            sorted_list = sorted(zipped, key=lambda x: x[1], reverse=True)
            sorted_rank = [sorted_list[i-1]+(i,) for i in range(1, len(sorted_list)+1)]
            # print(sorted_rank)

            new_row_weights = pd.DataFrame({
                    "dimension":[dimension]*len(sorted_rank),
                    "dataset":[dataset_names[i]]*len(sorted_rank),
                    "technique":[technique_names[j]]*len(sorted_rank),
                    "feature":[sorted_rank[i][0] for i in range(len(sorted_rank))],
                    "coefficent":[sorted_rank[i][1] for i in range(len(sorted_rank))],
                    "rank":[sorted_rank[i][2] for i in range(len(sorted_rank))]
                })
            feature_weights = pd.concat([feature_weights, new_row_weights])
# Remember to only add rows to dataset once and save files! 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.007219530978993305, tolerance: 0.006165457101344418
  model = cd_fast.enet_coordinate_descent_gram(
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.006677170334221216, tolerance: 0.006511736523578114
  model = cd_fast.enet_coordinate_descent_gram(


In [246]:
results_table.to_csv("/Users/simonrdkn/Documents/commalla_acoustic_analysis/final results/lin_reg_results.csv")
feature_weights.to_csv("/Users/simonrdkn/Documents/commalla_acoustic_analysis/final results/lin_reg_weights.csv")

# Lasso Regression 
lots of misc testing

In [213]:
# Loading and Preparing Data
sauter_feature_set = pd.read_csv("/Users/simonrdkn/Documents/commalla_acoustic_analysis/feature sets/sauter_feature_set.csv")
sauter_feature_set = sauter_feature_set.dropna()

X = sauter_feature_set[sauter_feature_set.columns[7:]]
print(X.columns)
y = sauter_feature_set.mean_arousal
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Index(['duration', 'mean_pitch', 'min_pitch', 'max_pitch', 'stdev_pitch',
       'mean_intensity', 'spectral_cog', 'spectral_sd', 'amp_rms',
       'sd_intensity'],
      dtype='object')


In [214]:
lasso = Lasso(alpha=0.01)
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print("Model Score: ", lasso.score(X_test, y_test))

Model Score:  0.7536980768604028


In [215]:
# Lasso 
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10,]}
lasso_cv = GridSearchCV(Lasso(), param_grid, cv=ShuffleSplit(n_splits=5, test_size=0.25))
lasso_cv.fit(X_train, y_train)
print("Best Parameters:", lasso_cv.best_params_)
print("Best Lasso Score (not test):", lasso_cv.best_score_)

lasso_cv = LassoCV(cv=5) # gridsearchCV does not return coef_ (required for weights info)
lasso_cv.fit(X_train, y_train)
lasso_cv.score(X_test, y_test) # coefficent of determination r^2

# Elastic Net
param_grid = {'l1_ratio': [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}
elastic_net_cv = GridSearchCV(ElasticNetCV(), param_grid, cv=5)
elastic_net_cv.fit(X_train, y_train)
best_l1 = elastic_net_cv.best_params_["l1_ratio"]
print("Elastic r^2 score:", elastic_net_cv.score(X_test, y_test))

Best Lasso Parameters: {'alpha': 0.01}
Best Lasso Score (not test): 0.7671381077906702
Elastic r^2 score: 0.753582939994628


In [217]:
zipped = list(zip(lasso_cv.coef_, X.columns)) # list(map(list, zip(lasso_cv.coef_, X.columns)))
sorted_list = sorted(zipped, key=lambda x: x[0], reverse=True)
sorted_rank = [sorted_list[i-1]+(i,) for i in range(1, len(sorted_list)+1)]
print(sorted_rank)



AttributeError: 'GridSearchCV' object has no attribute 'coef_'

Arousal r2 = 0.754 <br>
Valence r2 = 0.285
